In [1]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate

In [2]:
ge = pdr.get_data_yahoo('GE', datetime(2000, 1, 1), datetime(2018, 1, 1))
aapl = pdr.get_data_yahoo('AAPL', datetime(2000, 1, 1), datetime(2018, 1, 1))
fb = pdr.get_data_yahoo('FB', datetime(2012, 6, 6), datetime(2018, 1, 1))
gs = pdr.get_data_yahoo('GS', datetime(2000, 1, 1), datetime(2018, 1, 1))
btc = pdr.get_data_yahoo('BTC-USD', datetime(2011, 1, 1), datetime(2018, 1, 1))
fb.head()

In [3]:
#data preprocessing - dropping columns Close, Volume. Adj Close will be the Y variable
ge.drop(['Close','Volume'],axis =1, inplace = True)
aapl.drop(['Close','Volume'],axis =1, inplace = True)
fb.drop(['Close','Volume'],axis =1, inplace = True)
gs.drop(['Close','Volume'],axis =1, inplace = True)
btc.drop(['Close','Volume'],axis =1, inplace = True)
fb.head()

In [4]:
plt.plot(fb["Adj Close"])
display(plt.show())

In [5]:
#LSTMs are sensitive to data scales when activation functions are used.Its a good practise to range the data between 0,1
#Normalising using MinMaxscaler
#FB
fb_values = fb.values.astype('float32')
fb_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_fb = fb_scaler.fit_transform(fb_values)
print(scaled_fb)
#GE
ge_values = ge.values.astype('float32')
ge_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_ge = ge_scaler.fit_transform(ge_values)
#AAPL
aapl_values = aapl.values.astype('float32')
aapl_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_aapl = aapl_scaler.fit_transform(aapl_values)
#GS
gs_values = gs.values.astype('float32')
gs_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_gs = gs_scaler.fit_transform(gs_values)
#BTC
btc_values = btc.values.astype('float32')
btc_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_btc = btc_scaler.fit_transform(btc_values)

In [6]:
fb_cols,ge_cols,aapl_cols,gs_cols,btc_cols, cols_names = list(),list(),list(),list(),list(), list()
features = scaled_fb.shape[1] #Number of features. same for all
fb_df = pd.DataFrame(scaled_fb)
ge_df = pd.DataFrame(scaled_ge)
aapl_df = pd.DataFrame(scaled_aapl)
gs_df = pd.DataFrame(scaled_gs)
btc_df = pd.DataFrame(scaled_btc)

#Timestep is the number of previous time steps to be used as input to predict the next time step
#We can change values to see which one gives the best results for our model
input_ts = 25

#Based on the value of the timestep variable 'input_ts', values at t-n, ... t-1 are appended to the dataset 
#for all the features to create an input sequence
#Hence, one input sequence will have (input_ts * features) number of variables 
for i in range(input_ts, 0, -1):
  fb_cols.append(fb_df.shift(i))
  ge_cols.append(ge_df.shift(i))
  aapl_cols.append(aapl_df.shift(i))
  gs_cols.append(gs_df.shift(i))
  btc_cols.append(btc_df.shift(i))
  cols_names += [('feature%d(t-%d)' % (j+1, i)) for j in range(features)]
  
#Sequence (t) is also appended which will be our output timestep, ie, the value of timestep 't' is 
#predicted using the data for timesteps from (t-1) to (t-input_timeSteps)
fb_cols.append(fb_df.shift(0))
ge_cols.append(ge_df.shift(0))
aapl_cols.append(aapl_df.shift(0))
gs_cols.append(gs_df.shift(0))
btc_cols.append(btc_df.shift(0))

cols_names += [('feature%d(t)' % (j+1)) for j in range(features)]

fb_ModifiedSeq = concat(fb_cols, axis=1)
ge_ModifiedSeq = concat(ge_cols, axis=1)
aapl_ModifiedSeq = concat(aapl_cols, axis=1)
gs_ModifiedSeq = concat(gs_cols, axis=1)
btc_ModifiedSeq = concat(btc_cols, axis=1)
fb_ModifiedSeq .columns = cols_names
ge_ModifiedSeq .columns = cols_names
aapl_ModifiedSeq .columns = cols_names
gs_ModifiedSeq .columns = cols_names
btc_ModifiedSeq .columns = cols_names
fb_ModifiedSeq .dropna(inplace=True) # Omitting rows with any of the missing values
ge_ModifiedSeq .dropna(inplace=True)
aapl_ModifiedSeq .dropna(inplace=True)
gs_ModifiedSeq .dropna(inplace=True)
btc_ModifiedSeq .dropna(inplace=True)
print(fb_ModifiedSeq)

In [7]:
#####################################Training and Test data sets#############################################################
#We cannot use cross validation method here to validate our model because sequence is important in time series.##############
#Instead, we can split our past data into train data and test data.##########################################################

In [8]:
#Using 90% of the data for training and 10% for testing
fb_train_size = int(len(fb_ModifiedSeq.values) * 0.9)
fb_test_size = len(fb_ModifiedSeq.values) - fb_train_size
fb_train, fb_test = fb_ModifiedSeq.values[0:fb_train_size,:], fb_ModifiedSeq.values[fb_train_size:len(fb_ModifiedSeq.values),:]

ge_train_size = int(len(ge_ModifiedSeq.values) * 0.9)
ge_test_size = len(ge_ModifiedSeq.values) - ge_train_size
ge_train, ge_test = ge_ModifiedSeq.values[0:ge_train_size,:], ge_ModifiedSeq.values[ge_train_size:len(ge_ModifiedSeq.values),:]

aapl_train_size = int(len(aapl_ModifiedSeq.values) * 0.9)
aapl_test_size = len(aapl_ModifiedSeq.values) - aapl_train_size
aapl_train, aapl_test = aapl_ModifiedSeq.values[0:aapl_train_size,:], aapl_ModifiedSeq.values[aapl_train_size:len(aapl_ModifiedSeq.values),:]

gs_train_size = int(len(gs_ModifiedSeq.values) * 0.9)
gs_test_size = len(gs_ModifiedSeq.values) - gs_train_size
gs_train, gs_test = gs_ModifiedSeq.values[0:gs_train_size,:], gs_ModifiedSeq.values[gs_train_size:len(gs_ModifiedSeq.values),:]

btc_train_size = int(len(btc_ModifiedSeq.values) * 0.9)
btc_test_size = len(btc_ModifiedSeq.values) - btc_train_size
btc_train, btc_test = btc_ModifiedSeq.values[0:btc_train_size,:], btc_ModifiedSeq.values[btc_train_size:len(btc_ModifiedSeq.values),:]

print(len(fb_train), len(fb_test))

In [9]:
fb_train

In [10]:
#Out of the total ((input_ts * features) + features) columns added by the above steps, the first (input_ts * features) columns
#that correspond to the timesteps (t-1) to (t-input_ts) will be our input features. Our prediction variable,
#which is the last column (that corresponds to 'close' price at timestep (t)) will be our 'y' variable.
fb_trainX = fb_train[:, :features*input_ts] 
fb_trainY = fb_train[:,-1]
ge_trainX = ge_train[:, :features*input_ts] 
ge_trainY = ge_train[:,-1]
aapl_trainX = aapl_train[:, :features*input_ts] 
aapl_trainY = aapl_train[:,-1]
gs_trainX = gs_train[:, :features*input_ts] 
gs_trainY = gs_train[:,-1]
btc_trainX = btc_train[:, :features*input_ts] 
btc_trainY = btc_train[:,-1]
    
fb_testX = fb_test[:, :features*input_ts] 
fb_testY = fb_test[:,-1]
ge_testX = ge_test[:, :features*input_ts] 
ge_testY = ge_test[:,-1]
aapl_testX = aapl_test[:, :features*input_ts] 
aapl_testY = aapl_test[:,-1]
gs_testX = gs_test[:, :features*input_ts] 
gs_testY = gs_test[:,-1]
btc_testX = btc_test[:, :features*input_ts] 
btc_testY = btc_test[:,-1]
 
#Input is reshaped to 3D [Number of rows, timesteps, Number of features] . Input requirement for LSTM   
fb_trainX = np.reshape(fb_trainX,(fb_trainX.shape[0], input_ts,features))
ge_trainX = np.reshape(ge_trainX,(ge_trainX.shape[0], input_ts,features))
aapl_trainX = np.reshape(aapl_trainX,(aapl_trainX.shape[0], input_ts,features))
gs_trainX = np.reshape(gs_trainX,(gs_trainX.shape[0], input_ts,features))
btc_trainX = np.reshape(btc_trainX,(btc_trainX.shape[0], input_ts,features))

fb_testX = np.reshape(fb_testX,(fb_testX.shape[0], input_ts, features))
ge_testX = np.reshape(ge_testX,(ge_testX.shape[0], input_ts, features))
aapl_testX = np.reshape(aapl_testX,(aapl_testX.shape[0], input_ts, features))
gs_testX = np.reshape(gs_testX,(gs_testX.shape[0], input_ts, features))
btc_testX = np.reshape(btc_testX,(btc_testX.shape[0], input_ts, features))

In [11]:
############################################# Build and fit LSTM Model #############################################

In [12]:
#Build the LSTM model
#FB
fb_model = Sequential()
fb_model.add(LSTM(260, input_shape=(fb_trainX.shape[1], fb_trainX.shape[2]), return_sequences=True, activation='linear')) #stack 2 lstm s together
fb_model.add(LSTM(260, input_shape=(fb_trainX.shape[1], fb_trainX.shape[2]), return_sequences=False, activation='linear'))
fb_model.add(Dense(1))
fb_model.compile(loss='mse', optimizer='adam') #keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Fit our model on the Training Set
ret = fb_model.fit(fb_trainX, fb_trainY, epochs=100, batch_size=50, validation_split=0.2, verbose=2, shuffle=False)

In [13]:
#GE
ge_model = Sequential()
ge_model.add(LSTM(300, input_shape=(ge_trainX.shape[1], ge_trainX.shape[2]), return_sequences=True, activation='linear')) #stack 2 lstm s together
ge_model.add(LSTM(300, input_shape=(ge_trainX.shape[1], ge_trainX.shape[2]), return_sequences=False, activation='linear'))
ge_model.add(Dense(1))
ge_model.compile(loss='mse', optimizer='adam') #keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Fit our model on the Training Set
ge_ret = ge_model.fit(ge_trainX, ge_trainY, epochs=80, batch_size=200, validation_split=0.2, verbose=2, shuffle=False)

In [14]:
#AAPL
aapl_model = Sequential()
aapl_model.add(LSTM(350, input_shape=(aapl_trainX.shape[1], aapl_trainX.shape[2]), return_sequences=True, activation='linear')) #stack 2 lstm s together
aapl_model.add(LSTM(120, input_shape=(aapl_trainX.shape[1], aapl_trainX.shape[2]), return_sequences=False, activation='linear'))
aapl_model.add(Dense(1))
aapl_model.compile(loss='mse', optimizer='adam') #keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Fit our model on the Training Set
aapl_ret = aapl_model.fit(aapl_trainX, aapl_trainY, epochs=100, batch_size=250, validation_split=0.2, verbose=2, shuffle=False)

In [15]:
#GS
gs_model = Sequential()
gs_model.add(LSTM(265, input_shape=(gs_trainX.shape[1], gs_trainX.shape[2]), return_sequences=True, activation='linear')) #stack 2 lstm s together
gs_model.add(LSTM(260, input_shape=(gs_trainX.shape[1], gs_trainX.shape[2]), return_sequences=False, activation='linear'))
gs_model.add(Dense(1))
gs_model.compile(loss='mse', optimizer='adam') #keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Fit our model on the Training Set
gs_ret = gs_model.fit(gs_trainX, gs_trainY, epochs=100, batch_size=150, validation_split=0.2, verbose=2, shuffle=False)

In [16]:
#BTC
btc_model = Sequential()
btc_model.add(LSTM(280, input_shape=(btc_trainX.shape[1], btc_trainX.shape[2]), return_sequences=True, activation='linear')) #stack 2 lstm s together
btc_model.add(LSTM(280, input_shape=(btc_trainX.shape[1], btc_trainX.shape[2]), return_sequences=False, activation='linear'))
btc_model.add(Dense(1))
btc_model.compile(loss='mse', optimizer='adam') #keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Fit our model on the Training Set
btc_ret = btc_model.fit(btc_trainX, btc_trainY, epochs=100, batch_size=250, validation_split=0.2, verbose=2, shuffle=False)

In [17]:
#model.fit() method returns a History object. Its History.history attribute records training loss values and metrics values at successive epochs, as well as validation loss values and validation metrics values
#FB
fig = plt.figure()
plt.plot(ret.history['loss'],label='Train Loss')
plt.plot(ret.history['val_loss'],label='Test Loss')
plt.title('FB - Loss at successive epochs')
plt.legend(loc='best')
display(plt.show())
plt.close(fig)

#From the plot, we can see that the model has comparable performance on both train and validation datasets.

In [18]:
#GE
fig = plt.figure()
plt.plot(ge_ret.history['loss'],label='Train Loss')
plt.plot(ge_ret.history['val_loss'],label='Test Loss')
plt.title('GE - Loss at successive epochs')
plt.legend(loc='best')
display(plt.show())
plt.close(fig)

In [19]:
#AAPL
fig = plt.figure()
plt.plot(aapl_ret.history['loss'],label='Train Loss')
plt.plot(aapl_ret.history['val_loss'],label='Test Loss')
plt.title('AAPL - Loss at successive epochs')
plt.legend(loc='best')
display(plt.show())
plt.close(fig)

In [20]:
#GS
fig = plt.figure()
plt.plot(gs_ret.history['loss'],label='Train Loss')
plt.plot(gs_ret.history['val_loss'],label='Test Loss')
plt.title('GS - Loss at successive epochs')
plt.legend(loc='best')
display(plt.show())
plt.close(fig)

In [21]:
#BTC
fig = plt.figure()
plt.plot(btc_ret.history['loss'],label='Train Loss')
plt.plot(btc_ret.history['val_loss'],label='Test Loss')
plt.title('BTC - Loss at successive epochs')
plt.legend(loc='best')
display(plt.show())
plt.close(fig)

In [22]:
# We scaled our dataset before feeding to the LSTM. As a result, our prediction variable also has a scaled value. Therefore, we should invert it to the orginal form and then compare with the actual value to get the rmse. This will give us an error measurement in the same unit as the original variable.

#Make predictions for our test data
fb_predictions = fb_model.predict(fb_testX)
ge_predictions = ge_model.predict(ge_testX)
aapl_predictions = aapl_model.predict(aapl_testX)
gs_predictions = gs_model.predict(gs_testX)
btc_predictions = btc_model.predict(btc_testX)

In [23]:
fb_testY

In [24]:
#Invert the scales for predictions
fb_testX = fb_testX.reshape((fb_testX.shape[0], input_ts*features))
fb_combined1 = concatenate((fb_predictions, fb_testX[:, -3:]), axis=1)
fb_inverted_predictions = fb_scaler.inverse_transform(fb_combined1)[:,0]

ge_testX = ge_testX.reshape((ge_testX.shape[0], input_ts*features))
ge_combined1 = concatenate((ge_predictions, ge_testX[:, -3:]), axis=1)
ge_inverted_predictions = ge_scaler.inverse_transform(ge_combined1)[:,0]

aapl_testX = aapl_testX.reshape((aapl_testX.shape[0], input_ts*features))
aapl_combined1 = concatenate((aapl_predictions, aapl_testX[:, -3:]), axis=1)
aapl_inverted_predictions = aapl_scaler.inverse_transform(aapl_combined1)[:,0]

gs_testX = gs_testX.reshape((gs_testX.shape[0], input_ts*features))
gs_combined1 = concatenate((gs_predictions, gs_testX[:, -3:]), axis=1)
gs_inverted_predictions = gs_scaler.inverse_transform(gs_combined1)[:,0]

btc_testX = btc_testX.reshape((btc_testX.shape[0], input_ts*features))
btc_combined1 = concatenate((btc_predictions, btc_testX[:, -3:]), axis=1)
btc_inverted_predictions = btc_scaler.inverse_transform(btc_combined1)[:,0]

#Invert the scales for original data
fb_testY = fb_testY.reshape((len(fb_testY), 1))
fb_combined2 = concatenate((fb_testY, fb_testX[:, -3:]), axis=1)
fb_inverted_original = fb_scaler.inverse_transform(fb_combined2)[:,0]

ge_testY = ge_testY.reshape((len(ge_testY), 1))
ge_combined2 = concatenate((ge_testY, ge_testX[:, -3:]), axis=1)
ge_inverted_original = ge_scaler.inverse_transform(ge_combined2)[:,0]

aapl_testY = aapl_testY.reshape((len(aapl_testY), 1))
aapl_combined2 = concatenate((aapl_testY, aapl_testX[:, -3:]), axis=1)
aapl_inverted_original = aapl_scaler.inverse_transform(aapl_combined2)[:,0]

gs_testY = gs_testY.reshape((len(gs_testY), 1))
gs_combined2 = concatenate((gs_testY, gs_testX[:, -3:]), axis=1)
gs_inverted_original = gs_scaler.inverse_transform(gs_combined2)[:,0]

btc_testY = btc_testY.reshape((len(btc_testY), 1))
btc_combined2 = concatenate((btc_testY, btc_testX[:, -3:]), axis=1)
btc_inverted_original = btc_scaler.inverse_transform(btc_combined2)[:,0]


In [25]:

#Calculate RMSE
fb_rmse = sqrt(mean_squared_error(fb_inverted_original, fb_inverted_predictions))
print('FB - Test RMSE: %.3f' % fb_rmse)
ge_rmse = sqrt(mean_squared_error(ge_inverted_original, ge_inverted_predictions))
print('GE - Test RMSE: %.3f' % ge_rmse)
aapl_rmse = sqrt(mean_squared_error(aapl_inverted_original, aapl_inverted_predictions))
print('AAPL - Test RMSE: %.3f' % aapl_rmse)
gs_rmse = sqrt(mean_squared_error(gs_inverted_original, gs_inverted_predictions))
print('GS - Test RMSE: %.3f' % gs_rmse)
btc_rmse = sqrt(mean_squared_error(btc_inverted_original, btc_inverted_predictions))
print('BTC - Test RMSE: %.3f' % btc_rmse)

In [26]:
#FB
fig = plt.figure()
plt.plot(fb_inverted_predictions, color='red', label='Prediction')
plt.plot(fb_inverted_original, color='blue', label='Original')
plt.legend(loc='best')
plt.title("FB - Predictions Vs Original")
display(plt.show())
plt.close(fig)

In [27]:
#GE
fig = plt.figure()
plt.plot(ge_inverted_predictions, color='red', label='Prediction')
plt.plot(ge_inverted_original, color='blue', label='Original')
plt.legend(loc='best')
plt.title("GE - Predictions Vs Original")
display(plt.show())
plt.close(fig)

In [28]:
#AAPL
fig = plt.figure()
plt.plot(aapl_inverted_predictions, color='red', label='Prediction')
plt.plot(aapl_inverted_original, color='blue', label='Original')
plt.legend(loc='best')
plt.title("AAPL - Predictions Vs Original")
display(plt.show())
plt.close(fig)

In [29]:
#GS
fig = plt.figure()
plt.plot(gs_inverted_predictions, color='red', label='Prediction')
plt.plot(gs_inverted_original, color='blue', label='Original')
plt.legend(loc='best')
plt.title("GS - Predictions Vs Original")
display(plt.show())
plt.close(fig)

In [30]:
#BTC
fig = plt.figure()
plt.plot(btc_inverted_predictions, color='red', label='Prediction')
plt.plot(btc_inverted_original, color='blue', label='Original')
plt.legend(loc='best')
plt.title("BTC - Predictions Vs Original")
display(plt.show())
plt.close(fig)